In [50]:
import json
from web3 import Web3, HTTPProvider
from web3.contract import ConciseContract
from cryptography.fernet import Fernet
import pickle
from datetime import datetime
from eth_account.messages import encode_defunct

## publishing the contract

In [304]:
# web3.py instance
w3 = Web3(HTTPProvider("HTTP://127.0.0.1:7545"))
print(w3.isConnected())

key="a2f49f40901545cc550295884c15782652d2e847fdec80e4e58bde2a10c096da"
acct = w3.eth.account.privateKeyToAccount(key)

# compile your smart contract with truffle first
truffleFile = json.load(open(r'C:\Users\mo7amad mo3taz\Desktop\security project\python_edition\pythonDapp\build\contracts\Greeter.json'))
abi = truffleFile['abi']
bytecode = truffleFile['bytecode']
contract= w3.eth.contract(bytecode=bytecode, abi=abi)

#building transaction
construct_txn = contract.constructor().buildTransaction({
    'from': acct.address,
    'nonce': w3.eth.getTransactionCount(acct.address),
    'gas': 3000000,
    'gasPrice': w3.toWei('21', 'gwei')})

signed = acct.signTransaction(construct_txn)

tx_hash=w3.eth.sendRawTransaction(signed.rawTransaction)
print(tx_hash.hex())
tx_receipt = w3.eth.waitForTransactionReceipt(tx_hash)
contract_address = tx_receipt['contractAddress']
print("Contract Deployed At:", tx_receipt['contractAddress'])


True
0x51a756edd6cedcd36365fa3eaabf58154731029d2b90d66823eb1a83647b9244
Contract Deployed At: 0x91b463A56dDF6f31539966559121B6FD43CB748D


### getting a contract instance

In [305]:
truffleFile = json.load(open(r'C:\Users\mo7amad mo3taz\Desktop\security project\python_edition\pythonDapp\build\contracts\Greeter.json'))
abi = truffleFile['abi']
bytecode = truffleFile['bytecode']

w3 = Web3(HTTPProvider("HTTP://127.0.0.1:7545")) #modify
print(w3.isConnected())
contract_address = Web3.toChecksumAddress(contract_address) #modify
contract = w3.eth.contract(abi=abi, bytecode=bytecode)
contract_instance = w3.eth.contract(abi=abi, address=contract_address)
print(contract_instance.functions.greet().call())

True
Hello


### defining the clinic entity

In [306]:
def load_enc_key(enc_path):
        with open(enc_path, "rb") as infile:
 	         return pickle.load(infile)

def load_count(count_path):
        with open(count_path, "rb") as infile:
 	         return pickle.load(infile)

def write_count(path, count):
        with open(path, "wb") as outfile:
 	# "wb" argument opens the file in binary mode
	        pickle.dump(count, outfile)

In [287]:
def store_count(path,count):
    with open(path, "wb") as outfile:
	    pickle.dump(count, outfile)
    
def to_32byte_hex(val):
   return Web3.toHex(Web3.toBytes(val).rjust(32, b'\0'))

In [307]:
class clinic:
    def __init__ (self,clinic_num):
        
        if clinic_num == 0:
            self.address = '0xA18701FcEfcB34548e3F1168018072acFceD24A3'
            self.private_key = '69517d7170fbec9e1ca85ab44b89f1ee645d77ac05750d3e230078d1251561b2'
            self.enc_key = load_enc_key(r'C:\Users\mo7amad mo3taz\Desktop\security project\python_edition\c1_enc.pickle')
            self.count_path = r'C:\Users\mo7amad mo3taz\Desktop\security project\python_edition\c1_count.pickle'
            self.patient_count = load_count(self.count_path)
            self.v_vount_path = r'C:\Users\mo7amad mo3taz\Desktop\security project\python_edition\c1_v_count.pickle'
            self.v_count = load_count(self.count_path)

        elif clinic_num == 1:
            self.address = '0x76d4fCF227AF4A6464F093566eD980Dfe7227CB8'
            self.private_key = 'a2f49f40901545cc550295884c15782652d2e847fdec80e4e58bde2a10c096da'
            self.enc_key = load_enc_key(r'C:\Users\mo7amad mo3taz\Desktop\security project\python_edition\c2_enc.pickle')
            self.count_path = r'C:\Users\mo7amad mo3taz\Desktop\security project\python_edition\c2_count.pickle'
            self.patient_count = load_count(self.count_path)
            self.v_vount_path = r'C:\Users\mo7amad mo3taz\Desktop\security project\python_edition\c2_v_count.pickle'
            self.v_count = load_count(self.count_path)

        else:
            self.address = '0x042607F7A3CeD2E6014D0B23E869EDD5471a54bc'
            self.private_key = 'd54950b31dfe059def052b9d703703031835713540d256034e1920fa7ded6df1'
            self.enc_key = load_enc_key(r'C:\Users\mo7amad mo3taz\Desktop\security project\python_edition\c3_enc.pickle')
            self.count_path = r'C:\Users\mo7amad mo3taz\Desktop\security project\python_edition\c3_count.pickle'
            self.patient_count = load_count(self.count_path)
            self.v_vount_path = r'C:\Users\mo7amad mo3taz\Desktop\security project\python_edition\c3_v_count.pickle'
            self.v_count = load_count(self.count_path)

        self.encrypter_decrypter = Fernet(self.enc_key)
        


    def sign_packet(self, packet):
        msgHash = Web3.keccak(text = packet)
        msgHash = encode_defunct(msgHash)
        signed_message = w3.eth.account.sign_message(msgHash, private_key=self.private_key)
        print('signed_message: ',type(signed_message))
        return (signed_message.v, to_32byte_hex(signed_message.r), to_32byte_hex(signed_message.s))

    def add_new_patient(self, name, age, weight, height, sex, blood_p, pulse):
        now = datetime.now()
        dt_string = now.strftime("%d/%m/%Y %H:%M:%S") # time_stamp
        data = name +" # "+ age +" # "+ weight +" # "+ height +" # "+ sex +" # "+ blood_p +" # "+ pulse + " # " + dt_string
        data_enc = self.encrypter_decrypter.encrypt(data.encode())
        print('original data: ',data)
        print()
        print('enc data: ',data_enc)
        print()
        packet = str(self.patient_count)+ "#" + data_enc.decode('utf-8')
        self.patient_count +=1
        write_count(self.count_path,self.patient_count)
        print('packet: ',packet)
        print()
        signed_message = self.sign_packet(packet)
        hashed_message = Web3.keccak(text = packet)
        print('hashed message: ',hashed_message)
        print()
        tx_hash = contract_instance.functions.add_patient(packet, hashed_message, signed_message[0], signed_message[1], signed_message[2]).transact({
            'from': self.address,
            'nonce': w3.eth.getTransactionCount(self.address),
            'gas': 1728712,
            'gasPrice': w3.toWei('21', 'gwei')})
        w3.eth.waitForTransactionReceipt(tx_hash)
        # print('response: {}'.format(
        #     contract_instance.functions.greet().call()
        # ))
        print("success, patient id: ", self.patient_count-1)
        print('////////////////////////////////////')

    def regular_visit(self, patient_id, blood_p, pulse, reason, presc, follow_up):
            now = datetime.now()
            dt_string = now.strftime("%d/%m/%Y %H:%M:%S") # time_stamp
            data = blood_p +" # "+ pulse +" # "+ reason +" # "+ presc +" # "+ follow_up + " # " + dt_string + " # " + str(self.v_count)
            data_enc = self.encrypter_decrypter.encrypt(data.encode())
            print('original data: ',data)
            print()
            print('enc data: ',data_enc)
            print()
            packet = str(patient_id)+ "#" + data_enc.decode('utf-8')
            print('packet: ',packet)
            print()
            signed_message = self.sign_packet(packet)
            hashed_message = Web3.keccak(text = packet)
            print('hashed message: ',hashed_message)
            print()
            tx_hash = contract_instance.functions.add_reg_visit(packet, hashed_message, signed_message[0], signed_message[1], signed_message[2]).transact({
                'from': self.address,
                'nonce': w3.eth.getTransactionCount(self.address),
                'gas': 1728712,
                'gasPrice': w3.toWei('21', 'gwei')})
            w3.eth.waitForTransactionReceipt(tx_hash)
            # print('response: {}'.format(
            # contract_instance.functions.greet().call()
            # ))
            print('////////////////////////////////////')
            self.v_count+=1
            write_count(self.v_vount_path,self.v_count)

    def lab_visit(self, patient_id, blood_p, pulse, lab_test_typr, lab_test_result):
        now = datetime.now()
        dt_string = now.strftime("%d/%m/%Y %H:%M:%S") # time_stamp
        data = blood_p +" # "+ pulse +" # "+ lab_test_typr +" # "+ lab_test_result + " # " + dt_string + " # " + str(self.v_count)
        data_enc = self.encrypter_decrypter.encrypt(data.encode())
        print('original data: ',data)
        print()
        print('enc data: ',data_enc)
        print()
        packet = str(patient_id)+ "#" +data_enc.decode('utf-8')
        print('packet: ',packet)
        print()
        signed_message = self.sign_packet(packet)
        hashed_message = Web3.keccak(text = packet)
        print('hashed message: ',hashed_message)
        print()
        tx_hash = contract_instance.functions.add_lab_visit(packet, hashed_message, signed_message[0], signed_message[1], signed_message[2]).transact({
            'from': self.address,
            'nonce': w3.eth.getTransactionCount(self.address),
            'gas': 1728712,
            'gasPrice': w3.toWei('21', 'gwei')})
        w3.eth.waitForTransactionReceipt(tx_hash)
        # print('response: {}'.format(
        #     contract_instance.functions.greet().call()
        # ))
        print('////////////////////////////////////')
        self.v_count+=1
        write_count(self.v_vount_path,self.v_count)

    def view_records(self, patient_id):
        
        packet = patient_id
        print('packet: ',packet)
        print()
        signed_message = self.sign_packet(packet)
        print('signed message: ', signed_message)
        print()
        hashed_message = Web3.keccak(text = packet)
        print('hashed message: ',hashed_message)
        print()
        tx_hash = contract_instance.functions.define_records(packet, hashed_message, signed_message[0], signed_message[1], signed_message[2]).transact({
            'from': self.address,
            'nonce': w3.eth.getTransactionCount(self.address),
            'gas': 1728712,
            'gasPrice': w3.toWei('21', 'gwei')})
        w3.eth.waitForTransactionReceipt(tx_hash)
        all_records = contract_instance.functions.retrive_records().call()
        result = []
        for record in all_records:
            hed_dat = record.split("#")
            # print(hed_dat)
            if hed_dat[0] == patient_id:
                decrypted = self.encrypter_decrypter.decrypt(hed_dat[1].encode('utf-8'))
                result.append(decrypted.decode())
        print('records')
        print(result)


In [308]:
c1 = clinic(2)
c1.add_new_patient('pat1', '20', '150 kg', '100 cm', 'M', '110 pb', '200 gh')
c1.add_new_patient('pat2', '21', '140 kg', '110 cm', 'f', '110 pb', '200 gh')
c1.add_new_patient('pat3', '22', '130 kg', '120 cm', 'f', '110 pb', '200 gh')
# c1.view_records('6')
# c1.lab_visit('1', '1', '1', '1', '1')

original data:  pat1 # 20 # 150 kg # 100 cm # M # 110 pb # 200 gh # 06/06/2022 21:58:53

enc data:  b'gAAAAABinlx9QgZnfaltrQS12civNexZfUaBAd44h5XAb-YNPYgvPKynYDVS13hZFTIru-HBTsjfNit8pxVpJuVF_215LWEKyQbiXYm4CLpqwGvC_ZNYIz9JFGvO0gq842-kR7YSMPY8rxBNUFBAd0g9vjNlYghHvn9o8JznCw7qd3jeN4iIcp4='

packet:  0#gAAAAABinlx9QgZnfaltrQS12civNexZfUaBAd44h5XAb-YNPYgvPKynYDVS13hZFTIru-HBTsjfNit8pxVpJuVF_215LWEKyQbiXYm4CLpqwGvC_ZNYIz9JFGvO0gq842-kR7YSMPY8rxBNUFBAd0g9vjNlYghHvn9o8JznCw7qd3jeN4iIcp4=

signed_message:  <class 'eth_account.datastructures.SignedMessage'>
hashed message:  b'D\xd6\xd5\xd6r\xd5|\xacM\xb9>\xaf\xf1\x89r~\xa8\x8c(\xe7\xc4q\x13w\x02ys\xca\xae\xa1P\x08'

success, patient id:  0
////////////////////////////////////
original data:  pat2 # 21 # 140 kg # 110 cm # f # 110 pb # 200 gh # 06/06/2022 21:58:54

enc data:  b'gAAAAABinlx-rA1GAj0FSBBg4MXILBIjreHu3z0aSuKA3pBAdD25xs9sO2k16j8SB-Aommq3OCXDisv9olajJwLJqSz_B6ljVE8md3uQ-r5e-jC2l0yoZtrUc76PYyEyKvox-b29o2LsFpJK0WB6EF-beeIuKEe1ZVfa7aN_eSXR

In [312]:
c1.view_records('0')

packet:  0

signed_message:  <class 'eth_account.datastructures.SignedMessage'>
signed message:  (28, '0xb86b6664dec422a18fade9d9195799b87e5666b2d70db3d19a087e331700bdb0', '0x6fd1a502b9f07aba7b3915128e7bd3e579c2cac67ef05de648dc63a5fa14c3ab')

hashed message:  b'\x04HR\xb2\xa6p\xad\xe5@~x\xfb(c\xc5\x1d\xe9\xfc\xb9eB\xa0q\x86\xfe:\xed\xa6\xbb\x8a\x11m'

records
['pat1 # 20 # 150 kg # 100 cm # M # 110 pb # 200 gh # 06/06/2022 21:58:53', '1 # 1 # 1 # 1 # 1 # 06/06/2022 21:59:41 # 0', '1 # 1 # 1 # 1 # 1 # 06/06/2022 22:00:46 # 1']


In [311]:
c1.regular_visit('0', '1', '1', '1', '1', '1')

original data:  1 # 1 # 1 # 1 # 1 # 06/06/2022 22:00:46 # 1

enc data:  b'gAAAAABinlzu6T725VTiRtadXlQaZ32ld5tvrRPNhBTxrB54E1D4fzMg5ZGvMfno-PDUvrcl5uViblX4fbXgbuKtOGCkfkwId5Cc_u0z5Vykvy5ofDesmsVKMrGuV9QTQMkecXmado6c'

packet:  0#gAAAAABinlzu6T725VTiRtadXlQaZ32ld5tvrRPNhBTxrB54E1D4fzMg5ZGvMfno-PDUvrcl5uViblX4fbXgbuKtOGCkfkwId5Cc_u0z5Vykvy5ofDesmsVKMrGuV9QTQMkecXmado6c

signed_message:  <class 'eth_account.datastructures.SignedMessage'>
hashed message:  b'\x0c\x19\xac.w10\xff+\x01\xb4\xfb\x0f\x1d[\xb8\xfc\x82\xc6\xb4\xe8\xe6\x87\xf8\xca\xc7h2\x8e7\xee\x1e'

////////////////////////////////////
